In [1]:
import pandas as pd
from rapidfuzz import process, fuzz

# Sample data loading step
data = pd.read_csv('2008_2022_한국현대문학_인문_복합학_추가데이터_참고문헌_박완서.xlsx - Sheet1.csv')

nan_count = data['title'].isnull().sum()
print(f"Number of NaN values in the 'title' column: {nan_count}")

# Drop rows where 'title' or 'author' is NaN
#data = data.dropna(subset=['title', 'author'])

Number of NaN values in the 'title' column: 7


In [2]:
data.groupby('author').size().sort_values(ascending=False).head(10)

author
박완서    863
김윤식    120
이선미    110
김은하     89
김양선     80
권명아     72
김은정     56
최인욱     55
김미영     51
신수정     51
dtype: int64

In [3]:
# Create a dictionary to hold the best match for each title based on the author
title_mapping = {}

# Group data by author
for author, group in data.groupby('author'):
    # Check if the 'author' is NaN and continue to next iteration if so
    if pd.isna(author):
        continue

    titles = group['title'].dropna().unique()  # Drop NaN and get unique titles for this author

    for title in titles:
        # Generate a list of other titles by the same author to compare against
        other_titles = [t for t in titles if t != title]

        if other_titles:
            # Find the most similar title to the current one (excluding the same title)
            best_match, score, _ = process.extractOne(title, other_titles, scorer=fuzz.token_sort_ratio)
            
            # Apply a threshold to consider as a match
            if score > 70:  # threshold of 70 out of 100
                title_mapping[(author, title)] = best_match
            else:
                title_mapping[(author, title)] = title  # no similar title found, map to itself
        else:
            # If no other titles are available, map the title to itself
            title_mapping[(author, title)] = title

# Apply the mapping to the 'title' column, handle NaN in 'author' or 'title'
data['unified_title'] = data.apply(lambda row: title_mapping.get((row['author'], row['title']), row['title'] if pd.notna(row['title']) else pd.NA), axis=1)


In [4]:
# 'unified_title' 열에서 각 제목의 빈도 계산
title_counts = data['unified_title'].value_counts()

# 빈도가 2 이상인 제목만 필터링
repeated_titles = title_counts[title_counts > 1].index.tolist()

# 'title' 열이 repeated_titles 리스트에 해당하는 경우, 'unified_title' 열을 'title' 열의 값으로 설정
data['unified_title'] = data.apply(lambda row: row['title'] if row['title'] in repeated_titles else row['unified_title'], axis=1)

In [5]:
# Create a unique index for each unified title
unique_titles = data['unified_title'].unique()
title_to_index = {title: idx for idx, title in enumerate(unique_titles)}

# Map each 'unified_title' to its unique index
data['title_index'] = data['unified_title'].map(title_to_index)

In [6]:
data['unified_title'].value_counts()

unified_title
그 많던 싱아는 누가 다 먹었을까                                                  36
한국 현대 노년소설 연구                                                       34
박완서 문학 길찾기                                                          33
그 산이 정말 거기 있었을까                                                     25
소설, 노년을 말하다                                                         24
                                                                    ..
대한민국 아파트 발굴사                                                         1
김승옥 각색 시나리오에 내재된 '현대성'과 '여성의 주체구성'연구   -『태양을 훔친 여자』,『강변부인』을 중심으로     1
중산층의 몰락과 계급양극화                                                       1
과도기 의식과 멜로드라마적 상상력—김승옥의 <강변부인>에 대하여                                  1
학병(學兵)의 기억과 국가 - 1940년대 학병의 좌담회와 수기를 중심으로                            1
Name: count, Length: 8800, dtype: int64

In [8]:
# Save the results to a new CSV file
data.to_csv('updated_final_ouput.csv', encoding='utf-8-sig', index=False)